# 中文金融情绪词典构建流程

## 翻译

In [2]:
import pandas as pd
import numpy as np
import sys
import uuid
import requests
import hashlib
import time
from imp import reload
import json
import jieba
import jieba.posseg as pseg
from multiprocessing import Process, Manager,Pool
import multiprocessing
from tqdm import tqdm
import math
import os
from collections import Counter
from itertools import chain
import gensim

In [2]:
#定义使用有道智云翻译单词的函数
reload(sys)

YOUDAO_URL = 'https://openapi.youdao.com/api'
APP_KEY = '1b06ba3dbabdc53d'
APP_SECRET = '0ZzWnSfmcgrEpcldvcufautHXjwGIu5r'


def encrypt(signStr):
    hash_algorithm = hashlib.sha256()
    hash_algorithm.update(signStr.encode('utf-8'))
    return hash_algorithm.hexdigest()


def truncate(q):
    if q is None:
        return None
    size = len(q)
    return q if size <= 20 else q[0:10] + str(size) + q[size - 10:size]


def do_request(data):
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    return requests.post(YOUDAO_URL, data=data, headers=headers)


# def connect(word,fromlanguage,tolanguage):
def connect(word):
    try:
        q = word

        data = {}
        #设置翻译语种
        data['from'] = 'en'
        data['to'] = 'zh-CHS'
        data['signType'] = 'v3'
        curtime = str(int(time.time()))
        data['curtime'] = curtime
        salt = str(uuid.uuid1())
        signStr = APP_KEY + truncate(q) + salt + curtime + APP_SECRET
        sign = encrypt(signStr)
        data['appKey'] = APP_KEY
        data['q'] = q
        data['salt'] = salt
        data['sign'] = sign
        # data['vocabId'] = "您的用户词表ID"

        response = do_request(data)
        contentType = response.headers['Content-Type']
        if contentType == "audio/mp3":
            millis = int(round(time.time() * 1000))
            filePath = "合成的音频存储路径" + str(millis) + ".mp3"
            fo = open(filePath, 'wb')
            fo.write(response.content)
            fo.close()
        else:
            # print(response.content)
            # return response.content
            return json.loads(response.content.decode())['translation'][0]
    except:
        # print(word+'返回翻译失败')
        return np.nan



In [4]:
#读取LM情感词典的单词
LMdict=pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv',encoding='utf-8')
LMdictWordlist=LMdict['Word'].tolist()

In [5]:
#LM词典翻译为中文(单进程)
TranslatedWord=[]
for word in tqdm(LMdictWordlist[:1000]):
    t=connect(word)
    TranslatedWord.append([word,t])
TranslatedWord=pd.DataFrame(TranslatedWord,columns=['Word','Translation'])
print(TranslatedWord)


100%|██████████| 1000/1000 [02:41<00:00,  6.18it/s]

                 Word Translation
0            AARDVARK          土豚
1           AARDVARKS          土豚
2               ABACI          算盘
3               ABACK           吓
4              ABACUS          算盘
..                ...         ...
995    ADMINISTRATION          政府
996   ADMINISTRATIONS          政府
997    ADMINISTRATIVE          行政
998  ADMINISTRATIVELY          管理
999     ADMINISTRATOR         管理员

[1000 rows x 2 columns]


In [6]:
#读LM词典翻译为中文（多进程）
TranslatedWord=pd.DataFrame()
if __name__ == '__main__':
    with Pool(5) as p:
        t=LMdictWordlist[:1000]
        res=list(tqdm(p.imap(connect, t),total=len(t),desc='监视进度:'))
        # res=p.map(connect, LMdictWordlist[:10])
        wordlist=[]
        for j in range(len(t)):
            wordlist.append([t[j],res[j]])
        TranslatedWord=pd.DataFrame(wordlist,columns=['Word','Translation'])
    print(TranslatedWord)

监视进度:: 100%|██████████| 1000/1000 [00:31<00:00, 32.00it/s]

                 Word Translation
0            AARDVARK          土豚
1           AARDVARKS          土豚
2               ABACI          算盘
3               ABACK           吓
4              ABACUS          算盘
5            ABACUSES          算盘
6               ABAFT         在船尾
7             ABALONE          鲍鱼
8            ABALONES          鲍鱼
9             ABANDON          放弃
10          ABANDONED        被遗弃的
11         ABANDONING          放弃
12        ABANDONMENT          放弃
13       ABANDONMENTS          放弃
14           ABANDONS         抛弃了
15              ABASE          自卑
16             ABASED         自卑的
17          ABASEMENT          降低
18         ABASEMENTS          降低
19             ABASES          打倒
20              ABASH       使局促不安
21            ABASHED         尴尬的
22          ABASHEDLY         尴尬的
23            ABASHES       使局促不安
24           ABASHING       使局促不安
25          ABASHMENT          羞愧
26         ABASHMENTS          羞愧
27            ABASING          贬低
28            

In [4]:

#读取LM词典转化为中文（多进程）
TranslatedWord=pd.DataFrame()
if __name__ == '__main__':
    for i in range(int(len(LMdictWordlist)/10000)+1):    
        with Pool(5) as p:
            if (i+1)*10000>len(LMdictWordlist):
                t=LMdictWordlist[i*10000:]
            else:
                t=LMdictWordlist[i*10000:(i+1)*10000]
            res=list(tqdm(p.imap(connect, t),total=len(t),desc='监视进度:'))
            # res=p.map(connect, LMdictWordlist[:10])
            wordlist=[]
            for j in range(len(t)):
                wordlist.append([t[j],res[j]])
            TranslatedWord=pd.DataFrame(wordlist,columns=['Word','Translation'])
        TranslatedWord.to_excel('Translation'+str(i)+'.xlsx',encoding='utf-8')
    


监视进度:: 100%|██████████| 6531/6531 [06:10<00:00, 17.64it/s]


In [29]:
#合并所有翻译词到原表格
TranslatedWord=pd.DataFrame()
for i in range(9):
    t=pd.read_excel('情绪词典翻译结果/Translation'+str(i)+'.xlsx')
    TranslatedWord=TranslatedWord.append(t)
TranslatedWord=TranslatedWord.reset_index()[['Word','Translation']]
TranslatedWord=pd.merge(LMdict,TranslatedWord,how='left')
TranslatedWord.to_csv('情绪词典.csv',encoding='utf-8-sig')

## 分词

In [13]:
#加载停用词
with open('hit_stopwords.txt',encoding='utf-8') as f:
    words=f.readlines()
    sw=[i.replace('\n','') for i in words]
    sw.append(' ')
    sw.append('\n')

#生成大小写字母和数字
chars=[]
for i in range(97,123):
    chars.append(chr(i))
for i in range(65,91):
    chars.append(chr(i))
for i in range(10):
    chars.append(str(i))

#判断词语是否包含数字或字母
def isNumorAlphain(word):
    for i in chars:
        if i in word:
            return 1
    return 0

#分词函数
def split(t): 
    words = pseg.cut(t,use_paddle=True)
    tempwords=[]
    for word,flag in words:
        if word not in set(sw) and isNumorAlphain(word)==0:
            tempwords.append(word)
    return tempwords

#读取新闻文本
#开始分词
#pandas==0.24.0 tqdm==4.32.2
texts=pd.read_csv('news_all_company_ture.csv',encoding='utf-8')
texts['正文']=texts['正文'].astype(str)
# texts['分词']=texts.apply(lambda x:split(x['正文']),axis=1)
#分五组，避免表格过大
for i in range(0, len(texts), math.ceil(len(texts)/5)):
        if i+math.ceil(len(texts)/5)>len(texts):
            t=texts[i:]
            tqdm.pandas(desc="开始分词：")
            t['分词']=t.progress_apply(lambda x:split(str(x['正文'])), axis=1)
            t.to_csv('分词/'+str(i)+'.csv',encoding='utf-8-sig')
        else:
            t=texts[i:i+math.ceil(len(texts)/5)]
            tqdm.pandas(desc="开始分词：")
            t['分词']=t.progress_apply(lambda x:split(str(x['正文'])), axis=1)
            t.to_csv('分词/'+str(i)+'.csv',encoding='utf-8-sig')



开始分词：: 100%|██████████| 118398/118398 [23:50<00:00, 82.76it/s]  
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_5972/2243661295.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['分词']=t.progress_apply(lambda x:split(str(x['正文'])), axis=1)
开始分词：: 100%|██████████| 118398/118398 [31:21<00:00, 62.92it/s] 
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_5972/2243661295.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['分词']=t.progress_apply(lambda x:split(str(x['正文'])), axis=1)
开始分词：: 100%|██████████| 1

## 词频统计，调整词频表，生成word2vec模型预训练文本

In [31]:
#读取所有文本，进行词频统计
textsname=os.listdir('分词/')

#运用Counter计算词频的函数
def count_key_value(corpus):
    word_freq = Counter(chain(*corpus))
    return word_freq

#读取所有分词结果，进行词频统计
allwordfreq=pd.DataFrame()
for i in tqdm(textsname):
    corpus=pd.read_csv('分词/'+i)['分词'].tolist()[:]
    corpus=[corpus[c][2:-2].split("', '") for c in range(len(corpus))]
    tempwordfreq=count_key_value(corpus)
    tempwordfreq=pd.DataFrame([[key,value] for key,value in tempwordfreq.items()],columns=['word','times'])
    allwordfreq=allwordfreq.append(tempwordfreq)

allwordfreq=allwordfreq.groupby('word').apply(lambda x:sum(x['times']))
allwordfreq=pd.DataFrame({"word":allwordfreq.index,"times":allwordfreq.values})
allwordfreq.to_excel('初步分词词频表.xlsx')

100%|██████████| 5/5 [00:57<00:00, 11.48s/it]


In [32]:
#根据初步的词频表，机器加上人工审核去除特殊字符和不合理的词语，整理得到较为干净的语料准备进行word2vec训练
allwordfreq=pd.read_excel('初步分词词频表.xlsx')
allwordfreq['word']=allwordfreq['word'].astype(str)

#去除包含非中文字符部分
def is_chinese(word):
    if word == '':
        return 0
    for i in word:
        if i < '\u4e00' or i > '\u9fa5':
            return 0
    return 1

allwordfreq['is_chinese']=allwordfreq.apply(lambda x:is_chinese(x['word']),axis=1)
allwordfreq=allwordfreq[allwordfreq.is_chinese!=0][['word','times']].dropna().reset_index()
del allwordfreq['index']
print(allwordfreq)


        word  times
0        一一二     46
1        一一例      1
2       一一列举      9
3        一一对      2
4       一一对应     49
...      ...    ...
360654    龟鹿      3
360655   龟鹿丹      1
360656   龟鹿宁      1
360657    龟龄    187
360658     龠      2

[360659 rows x 2 columns]


In [33]:
#删除词频小于5的词
allwordfreq=allwordfreq[allwordfreq.times>5].reset_index()
del allwordfreq['index']
print(allwordfreq)

        word  times
0        一一二     46
1       一一列举      9
2       一一对应     49
3         一丁     11
4        一丁点      8
...      ...    ...
140919    龟山      9
140920    龟甲     28
140921    龟苓      6
140922   龟苓膏     67
140923    龟龄    187

[140924 rows x 2 columns]


In [34]:
#删除单个字
def is_single(word):
    if len(word)==1:
        return 1
    return 0

allwordfreq['is_single']=allwordfreq.apply(lambda x:is_single(x['word']),axis=1)
allwordfreq=allwordfreq[allwordfreq.is_single==0][['word','times']].dropna().reset_index()
del allwordfreq['index']
print(allwordfreq)


        word  times
0        一一二     46
1       一一列举      9
2       一一对应     49
3         一丁     11
4        一丁点      8
...      ...    ...
137057    龟山      9
137058    龟甲     28
137059    龟苓      6
137060   龟苓膏     67
137061    龟龄    187

[137062 rows x 2 columns]


In [36]:
#更细致的可以考虑删除叠词、根据词性筛选、进行命名实体识别

#保存筛选后的词频表
allwordfreq.to_csv('筛选后词频表.csv',encoding='utf-8-sig')

In [ ]:
#使用set数据类型加快判断
allwordfreq=set(allwordfreq['word'].drop_duplicates().tolist())

In [14]:
#根据数据清理后的词频表，生成进行word2vec的训练的语料
word2vec_corpus=[]
for i in os.listdir('分词'):
    t=pd.read_csv('分词/'+i)
    for text in tqdm(t['分词'].tolist()):
        words=[]
        for word in text[2:-2].split("', '"):
            if word in allwordfreq:
                words.append(word)
        # print(words)
        word2vec_corpus.append(words)
    print(len(word2vec_corpus))


100%|██████████| 118398/118398 [00:04<00:00, 24675.69it/s]


118398


100%|██████████| 118398/118398 [00:07<00:00, 16507.23it/s]


236796


100%|██████████| 118398/118398 [00:08<00:00, 14634.48it/s]


355194


100%|██████████| 118398/118398 [00:05<00:00, 19838.01it/s]


473592


100%|██████████| 118395/118395 [00:04<00:00, 25639.68it/s]

591987


## 进行模型训练

In [24]:
#word2vec训练
model=gensim.models.Word2Vec(word2vec_corpus,vector_size=300,window=5,min_count=5,sg=1,alpha=0.0005,epochs=20,workers=multiprocessing.cpu_count())
#word2vec模型保存
model.save('金融情绪词典word2vec_model.model')

In [4]:
#读取训练模型结果
model=gensim.models.Word2Vec.load('金融情绪词典word2vec_model.model')
#训练结果展示
print(model.wv.most_similar('研发',topn=10))

[('科研', 0.7208125591278076), ('产品开发', 0.7160720229148865), ('研制', 0.7090649008750916), ('产业化', 0.6690207123756409), ('技术开发', 0.6645838618278503), ('投入', 0.6575615406036377), ('技术创新', 0.6550519466400146), ('市场推广', 0.6383605003356934), ('核心技术', 0.6297745704650879), ('自主', 0.6281081438064575)]


In [31]:
#生成与词典中最相似的top5词典
df = []
for i in tqdm(pd.read_csv('情绪词典.csv')['Translation'].drop_duplicates().tolist()):
    try:
        res=model.wv.most_similar(i,topn=5)
        for j in res:
            df.append([i,j[0],j[1]])
    except:
        pass
df=pd.DataFrame(df,columns=['','seedword','word','similarity'])
df.to_csv('情绪词典_Word2vec_top5.csv',encoding='utf-8-sig')

100%|██████████| 42741/42741 [01:36<00:00, 442.51it/s] 
